## Advent of Code 2019

https://adventofcode.com/2019

In [10]:
from pathlib import Path
import numpy as np

In [52]:
input_dir = Path('./inputs')
def load_input(day_num, sep):
    with open(input_dir/f'input{day_num}.txt', 'r') as f:
        data = f.read().split(sep)[:-1]
    return [dp.strip() for dp in data]

### Day 1

In [54]:
data = load_input(1, '\n')
data = [int(dp) for dp in data]
data[0:2]

[90859, 127415]

#### Part 1

In [55]:
def fuel_fn(mass):
    return int(np.floor(mass/3) - 2)

In [56]:
assert(fuel_fn(12)) == 2
assert(fuel_fn(14)) == 2
assert(fuel_fn(1969)) == 654
assert(fuel_fn(100756)) == 33583

**Answer**

In [57]:
np.sum([fuel_fn(mass) for mass in data])

3262991

#### Part 2

In [34]:
def total_fuel_fn(initial_mass):
    total = 0
    req = fuel_fn(initial_mass)
    total += req
    while req > 0:
        req = max(fuel_fn(req), 0)
        total += req
    return total

In [35]:
assert(total_fuel_fn(14)) == 2
assert(total_fuel_fn(1969)) == 966
assert(total_fuel_fn(100756)) == 50346

**Answer**

In [36]:
np.sum([total_fuel_fn(initial_mass) for initial_mass in data])

4891620

### Day 2

In [102]:
data = load_input(2, ',')
data = [int(dp) for dp in data]
data[0:2]

[1, 0]

#### Part 1

In [128]:
opcode_slice_size = 4

def eval_instruction(inputs, instruction):
    if instruction == 1:
        return np.sum(inputs)
    elif instruction == 2:
        return np.product(inputs)

def eval_opcode_slice(intcode_program, start_index, termination_flag):
    opcode_slice = intcode_program[start_index:start_index+opcode_slice_size]
    instruction = opcode_slice[0]
    if instruction == 99:
        termination_flag = True
        return intcode_program, termination_flag
    inputs = [intcode_program[idx] for idx in opcode_slice[1:3]]
    output_loc = opcode_slice[3]
    eval_result = eval_instruction(inputs, instruction)
    intcode_program[output_loc] = eval_result
    return intcode_program, termination_flag

def eval_opcode(intcode_program):
    termination_flag = False
    start_index = 0
    while termination_flag == False:
        intcode_program, termination_flag = eval_opcode_slice(intcode_program, start_index, termination_flag)
        start_index += opcode_slice_size
    return intcode_program 

In [115]:
prog = [2,4,4,5,99,0]
res = eval_opcode(prog)

Terminating program


In [119]:
data[1:3] = [12, 2]

**Answer**

In [121]:
res = eval_opcode(data.copy())
res[0]

Terminating program


8017076

#### Part 2

In [122]:
def test_inputs(noun, verb):
    prog = data.copy()
    prog[1:3] = [noun, verb]
    res = eval_opcode(prog)
    return res[0]

In [130]:
target = 19690720
endpoint = 100
for noun in range(endpoint):
    for verb in range(endpoint):
        res = test_inputs(noun, verb)
        if res == target:
            print(noun, verb)
            print(100 * noun + verb)
            break

31 46
3146
